#### 一. LDA中要用到的公式  
1. 完全条件公式 :  
 $$p\left( { x }_{ j }|{ x }_{ \neg j } \right) =\frac { p\left( { x }_{ 1 },{ x }_{ 2 },...,{ x }_{ n } \right)  }{ p\left( { x }_{ j }|{ x }_{ 1 },{ x }_{ 2 },...{ x }_{ j-1 },{ x }_{ j+1 },...,{ x }_{ n } \right)  } $$
2. $\varphi$ : $k*v$阶矩阵, ${ \overrightarrow { { \varphi  }_{ k } }  }$是主题$k$上的词分布  
$\theta$ : $k*m$阶矩阵, $\overrightarrow { { \theta  }_{ m } } $: 第$m$篇文章的主体分布
3. 
4.   


#### 一. LDA整体思路
由于$P\left( \overrightarrow { z } |\overrightarrow { w }  \right)$不好求, 因此使用`collapse gibbs sampling`抽样, 利用完全条件公式(1),写出$p\left( { z }_{ i }|\overrightarrow { { z }_{ \neg i } } ,\overrightarrow { w }  \right) $. 用此公式模拟$P\left( \overrightarrow { z } |\overrightarrow { w }  \right)$
1. 写出LDA下整个文本集生成的联合概率   
 其中, $W$是语料库中所有单词的数量, 这些词单词, 在$z_i=k$的条件下, 在多次独立的'多项分布'试验中被抽取出来
$$p\left( \overrightarrow { w } ,\overrightarrow { z } |\overrightarrow { \alpha  } ,\overrightarrow { \beta  }  \right) =p\left( \overrightarrow { w } |\overrightarrow { z } ,\overrightarrow { \beta  }  \right) *p\left( \overrightarrow { z } |\overrightarrow { \alpha  }  \right) $$  
由于第一项不包含$\overrightarrow { \alpha  } $, 第二项不包括$\overrightarrow { \beta  } $, 所以可以分开处理  
  1. 第一项 : $$p\left( \overrightarrow { w } |\overrightarrow { z } ,\varphi  \right) =\prod _{ i=1 }^{ W }{ p\left( { w }^{ i }|{ z }^{ i } \right)  } =\prod _{ i=1 }^{ W }{ { \varphi  }_{ { z }_{ i },{ w }_{ i } } } $$其中, ${ \varphi  }_{ { z }_{ i },{ w }_{ i } }$表示在主题${ z }_{ i }$下, 得到单词${ w }_{ i }$的概率, $W$为语料中的单词数, ${ n }_{ k }^{ (t) }$为主题k下单词t被观测到的次数  
  而当我们${ \varphi  }_{ { z }_{ i },{ w }_{ i } }$拆成两项, 一个`over topic`, 另一个`over vocabulary` , 就有$$p\left( \overrightarrow { w } |\overrightarrow { z } ,\varphi  \right) =\prod _{ i=1 }^{ W }{ p\left( { w }^{ i }|{ z }^{ i } \right)  } =\prod _{ k=1 }^{ K }{ \prod _{ \{ i,{ z }_{ i }=k\}  }^{ W }{ p\left( { w }_{ i }=t|{ z }_{ i }=k \right)  }  } =\prod _{ k=1 }^{ K }{ \prod _{ t=1 }^{ V }{ { { \varphi  }_{ k,t } }^{ { n }_{ k }^{ (t) } } }  } $$得到上式的一些解释 :  
    1. 我们没有考虑超参数$\overrightarrow { \beta  } $, 而是先假设已知所有主题的词分布$\varphi_i $, 然后利用超参数$\overrightarrow { \beta  } $与$\varphi_i $的关系, 把$\varphi$积分掉. 推到如下 
    2. $p\left( \overrightarrow { w } |\overrightarrow { z } ,\overrightarrow { \beta  }  \right) =\int { p\left( \overrightarrow { w } |\overrightarrow { z } ,\varphi  \right) *p\left( \varphi |\overrightarrow { \beta  }  \right)  } d\varphi \\ \quad \quad \quad \quad \quad =\int { p\left( \overrightarrow { w } |\overrightarrow { z } ,\varphi  \right) *Dirichlet\left( \varphi |\overrightarrow { \beta  }  \right)  } d\varphi \\ \quad \quad \quad \quad \quad =\int { \prod _{ k=1 }^{ K }{ \prod _{ t=1 }^{ V }{ { { \varphi  }_{ k,t } }^{ { n }_{ k }^{ (t) } } }  } *\prod _{ k=1 }^{ K }{ \left( \frac { 1 }{ \triangle ({ \overrightarrow { \beta  }  }) } \prod _{ t=1 }^{ V }{ { { \varphi  }_{ k,t } }^{ { \beta  }_{ t }-1 } }  \right)  }  } d\varphi \\ \quad \quad \quad \quad \quad =\prod _{ k=1 }^{ K }{ \left[ \int { \left( \prod _{ t=1 }^{ V }{ { { \varphi  }_{ k,t } }^{ { n }_{ k }^{ (t) } } } *\frac { 1 }{ \triangle (\overrightarrow { \beta  } ) } \prod _{ t=1 }^{ V }{ { { \varphi  }_{ k,t } }^{ { \beta  }_{ t }-1 } }  \right)  } d\overrightarrow { { \varphi  }_{ k } }  \right]  } \\ \quad \quad \quad \quad \quad =\prod _{ k=1 }^{ K }{ \left[ \int { \left( \prod _{ t=1 }^{ V }{ { { \varphi  }_{ k,t } }^{ { n }_{ k }^{ (t) }+{ \beta  }_{ t }-1 } } *\frac { 1 }{ \triangle (\overrightarrow { \beta  } ) }  \right)  } d\overrightarrow { { \varphi  }_{ k } }  \right]  } \\ \quad \quad \quad \quad \quad =\prod _{ k=1 }^{ K }{ \frac { \triangle \left( \overrightarrow { { n }_{ k } } +\overrightarrow { \beta  }  \right)  }{ \triangle (\overrightarrow { \beta  } ) }  } ,\quad 其中\overrightarrow { { n }_{ k } } ={ \left\{ { n }_{ k }^{ (t) } \right\}  }_{ t=1 }^{ V }$  
    至此, 我们发现, 第一项的计算结果, 只和超参数$\overrightarrow { \beta  } $和$\overrightarrow { w } $的单词观测次数有关; 每个主题的单词分布$\overrightarrow { { \varphi  }_{ k } } $已在计算过程中被积分掉.   
    此过程, 属于$Dirichlet-Multinomial$共轭结构. 而$p\left( \overrightarrow { z } |\theta  \right) $的计算也遵循$Dirichlet-Multinomial$共轭结构  
  2. 

#### 一. LDA整体思路
由于$P\left( \overrightarrow { z } |\overrightarrow { w }  \right)$不好求, 因此使用`collapse gibbs sampling`抽样, 利用完全条件公式(1),写出$p\left( { z }_{ i }|\overrightarrow { { z }_{ \neg i } } ,\overrightarrow { w }  \right) $. 用此公式模拟$P\left( \overrightarrow { z } |\overrightarrow { w }  \right)$
1. 写出LDA下整个文本集生成的联合概率   
$$p\left( \overrightarrow { w } ,\overrightarrow { z } |\overrightarrow { \alpha  } ,\overrightarrow { \beta  }  \right) =p\left( \overrightarrow { w } |\overrightarrow { z } ,\overrightarrow { \beta  }  \right) *p\left( \overrightarrow { z } |\overrightarrow { \alpha  }  \right) $$ 
$\overrightarrow { w }$是真个语料库中观测到的词向量, $\overrightarrow { z }$是每个单词的主题分布
由于第一项不包含$\overrightarrow { \alpha  } $, 第二项不包括$\overrightarrow { \beta  } $, 所以可以分开处理  
  1. 第一项 : $$p\left( \overrightarrow { w } |\overrightarrow { z } ,\varphi  \right) =\prod _{ i=1 }^{ W }{ p\left( { w }^{ i }|{ z }^{ i } \right)  } =\prod _{ i=1 }^{ W }{ { \varphi  }_{ { z }_{ i },{ w }_{ i } } } $$其中,$W$是语料库中所有单词的数量, 这些词单词, 在$z_i=k$的条件下, 在多次独立的'多项分布'试验中被抽取出来. ${ \varphi  }_{ { z }_{ i },{ w }_{ i } }$表示在主题${ z }_{ i }$下, 得到单词${ w }_{ i }$的概率, $W$为语料中的单词数, ${ n }_{ k }^{ (t) }$为主题k下单词t被观测到的次数  
  而当我们${ \varphi  }_{ { z }_{ i },{ w }_{ i } }$拆成两项, 一个`over topic`, 另一个`over vocabulary` , 就有$$p\left( \overrightarrow { w } |\overrightarrow { z } ,\varphi  \right) =\prod _{ i=1 }^{ W }{ p\left( { w }^{ i }|{ z }^{ i } \right)  } =\prod _{ k=1 }^{ K }{ \prod _{ \{ i,{ z }_{ i }=k\}  }^{ W }{ p\left( { w }_{ i }=t|{ z }_{ i }=k \right)  }  } =\prod _{ k=1 }^{ K }{ \prod _{ t=1 }^{ V }{ { { \varphi  }_{ k,t } }^{ { n }_{ k }^{ (t) } } }  } $$得到上式的一些解释 :  
    1. 我们没有考虑超参数$\overrightarrow { \beta  } $, 而是先假设已知所有主题的词分布$\varphi_i $, 然后利用超参数$\overrightarrow { \beta  } $与$\varphi_i $的关系, 把$\varphi$积分掉. 推到如下 
    2. $p\left( \overrightarrow { w } |\overrightarrow { z } ,\overrightarrow { \beta  }  \right) =\int { p\left( \overrightarrow { w } |\overrightarrow { z } ,\varphi  \right) *p\left( \varphi |\overrightarrow { \beta  }  \right)  } d\varphi \\ \quad \quad \quad \quad \quad =\int { p\left( \overrightarrow { w } |\overrightarrow { z } ,\varphi  \right) *Dirichlet\left( \varphi |\overrightarrow { \beta  }  \right)  } d\varphi \\ \quad \quad \quad \quad \quad =\int { \prod _{ k=1 }^{ K }{ \prod _{ t=1 }^{ V }{ { { \varphi  }_{ k,t } }^{ { n }_{ k }^{ (t) } } }  } *\prod _{ k=1 }^{ K }{ \left( \frac { 1 }{ \triangle ({ \overrightarrow { \beta  }  }) } \prod _{ t=1 }^{ V }{ { { \varphi  }_{ k,t } }^{ { \beta  }_{ t }-1 } }  \right)  }  } d\varphi \\ \quad \quad \quad \quad \quad =\prod _{ k=1 }^{ K }{ \left[ \int { \left( \prod _{ t=1 }^{ V }{ { { \varphi  }_{ k,t } }^{ { n }_{ k }^{ (t) } } } *\frac { 1 }{ \triangle (\overrightarrow { \beta  } ) } \prod _{ t=1 }^{ V }{ { { \varphi  }_{ k,t } }^{ { \beta  }_{ t }-1 } }  \right)  } d\overrightarrow { { \varphi  }_{ k } }  \right]  } \\ \quad \quad \quad \quad \quad =\prod _{ k=1 }^{ K }{ \left[ \int { \left( \prod _{ t=1 }^{ V }{ { { \varphi  }_{ k,t } }^{ { n }_{ k }^{ (t) }+{ \beta  }_{ t }-1 } } *\frac { 1 }{ \triangle (\overrightarrow { \beta  } ) }  \right)  } d\overrightarrow { { \varphi  }_{ k } }  \right]  } \\ \quad \quad \quad \quad \quad =\prod _{ k=1 }^{ K }{ \frac { \triangle \left( \overrightarrow { { n }_{ k } } +\overrightarrow { \beta  }  \right)  }{ \triangle (\overrightarrow { \beta  } ) }  } ,\quad 其中\overrightarrow { { n }_{ k } } ={ \left\{ { n }_{ k }^{ (t) } \right\}  }_{ t=1 }^{ V }$  
    至此, 我们发现, 第一项的计算结果, 只和超参数$\overrightarrow { \beta  } $和$\overrightarrow { w } $的单词观测次数有关; 每个主题的单词分布$\overrightarrow { { \varphi  }_{ k } } $已在计算过程中被积分掉.   
    此过程, 属于$Dirichlet-Multinomial$共轭结构. 而$p\left( \overrightarrow { z } |\theta  \right) $的计算也遵循$Dirichlet-Multinomial$共轭结构  
  2. 

第二项 : $p\left( \overrightarrow { z } |\overrightarrow { \alpha  }  \right) $的计算也按照第一步的方法. 先假设由超参数$\overrightarrow { \alpha  } $确定的$\overrightarrow { \theta  } $已知, 转而计算$$   
$p\left( \overrightarrow { z } |\theta  \right) $